In [1]:
### Constants ###

# System specs
hybrid_system = True    #T/F, False: off-grid
batt_ch_rating = 5      #kW dc
batt_dc_rating = -10    #kW dc
inv_rating = 100        #kW ac, output
grid_ch_rating = 50     #kW dc, output
soc_low = 0.5           #%
batt_capacity = 100     #kWh
time_inc = 0.5          #hrs

# Grid connection
grid_export_rating = 1000   #kW ac
grid_import_rating = 1000   #kW ac

# Efficiency
e_pv = 0.98
e_inv = 0.95
e_gc = 0.95
e_bch = 0.95
e_bdc = 0.95

In [2]:
# Inputs (vary with each time-stamp)
solar_input = 88.90750134    #kW dc
load = 0           #kW ac
batt_soc = 0.5475      #%
grid_on = True         #T/F

In [3]:
# Battery status calculations (common)
batt_high = batt_soc > soc_low

batt_ch_max_to_full = (1-batt_soc) * batt_capacity / time_inc / e_bch
batt_ch_max = min(batt_ch_max_to_full, batt_ch_rating)

batt_dc_max_to_empty = -batt_soc * batt_capacity * e_bdc / time_inc
batt_dc_max = max(batt_dc_max_to_empty, batt_dc_rating)

# Solar 
solar_limited = min(solar_input,inv_rating/e_inv/e_pv)
solar_wasted_inv_limit = solar_input - solar_limited
print('to fix: solar_wasted_inv_limit needs to factor in battery charging')

to fix: solar_wasted_inv_limit needs to factor in battery charging


In [4]:
# System operating modes
if hybrid_system:
    if grid_on:
        grid_export_max = grid_export_rating   #kW ac
        grid_import_max = grid_import_rating   #kW ac
        inverter_on = True
    else:   #power cut
        grid_export_max = 0 
        grid_import_max = 0
        inverter_on = True
else:  #off-grid system
    if grid_on and not(batt_high):
        grid_export_max = 0 
        grid_import_max = grid_import_rating
        inverter_on = False
    else:
        grid_export_max = 0 
        grid_import_max = 0
        inverter_on = True

In [5]:
# Solar balance, at solar_input, kW dc
solar_for_load = min(load/e_inv/e_pv, solar_limited) * inverter_on
solar_for_batt = min(solar_limited-solar_for_load, batt_ch_max/e_pv)
solar_for_grid = min(solar_limited-solar_for_load-solar_for_batt, grid_export_max)
solar = solar_for_load + solar_for_batt + solar_for_grid
solar_wasted = solar_limited - solar

In [6]:
# Load balance, at load, kW ac
load_from_solar = solar_for_load*e_pv*e_inv
load_from_batt_initial = min(-batt_dc_max, load - load_from_solar)*batt_high
load_from_grid = min(grid_import_max, load - load_from_batt_initial - load_from_solar)
load_from_batt = load - load_from_grid - load_from_solar

In [7]:
# Battery balance, at battery input, kW dc
# Charging
batt_from_solar = solar_for_batt*e_pv
if batt_high:
    batt_ch_target = min(batt_ch_max,batt_from_solar)
else:
    batt_ch_target = batt_ch_max
batt_from_grid = min(batt_ch_target - batt_from_solar, grid_ch_rating, (grid_import_max - load_from_grid)*e_gc)
batt_ch = batt_from_solar + batt_from_grid

# Discharging
batt_dc = -load_from_batt/e_inv

# Net
if batt_ch > 0:
    batt = batt_ch
    batt_internal = batt_ch*e_bch
else:
    batt = batt_dc
    batt_internal = batt_dc/e_bdc

batt_energy_delta = batt_internal * time_inc
batt_soc_new = batt_soc + batt_energy_delta / batt_capacity


In [8]:
# Grid balance, at grid, ac
grid_from_solar = solar_for_grid * e_pv * e_inv
grid_for_load = load_from_grid
grid_for_batt = batt_from_grid/e_gc
grid = grid_for_batt + grid_for_load + grid_from_solar


In [9]:
# Losses
inverter_power_out = grid_from_solar + load_from_solar + load_from_batt
inverter_power_in = inverter_power_out/e_inv

loss_pv = solar*(1-e_pv)
loss_gc = grid_for_batt*(1-e_gc)
loss_inv = inverter_power_in*(1-e_inv)
loss_batt = batt - batt_internal
loss_total = loss_pv + loss_gc + loss_inv + loss_batt

In [10]:
# Zero-sum checks
# Load
load_check = load - load_from_solar - load_from_batt - load_from_grid
if load_check != 0:
    raise Exception("load zero-check failed")

# Solar
solar_check = solar_input - solar_wasted_inv_limit - solar_for_load - solar_for_batt - solar_for_grid - solar_wasted
if solar_check != 0:
    raise Exception("solar zero-check failed")

# Battery
if batt < 0:
    batt_check = batt_dc - loss_batt - batt_internal
else:
    batt_check = batt_ch - loss_batt - batt_internal
if batt_check != 0:
    raise Exception("batt zero-check failed")

# Grid
grid_check = grid - grid_for_batt - grid_for_load - grid_from_solar
if grid_check != 0:
    raise Exception("grid zero-check failed")

system_check = solar + grid - load - batt_internal - loss_total
if abs(system_check) > 0.000001:
    print(system_check)
    raise Exception("system zero-check failed")

156.04576749507999


Exception: system zero-check failed